En el siguiente notebook vamos a trabajar con PySpark. En él, vamos a usar las transformaciones y las acciones para analizar un pequeño conjunto de datos.

El primer paso que debemos dar es conectar Google Colab con nuestro Google Drive. Para ello, lanzaremos el siguiente trozo de código.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


Una vez hemos conectado Google Drive, pasamos a instalar Apache Spark en nuestro notebook. En concreto, vamos a trabajar con la versión 3.0.3.

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
!wget -q http://apache.mirrors.pair.com/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [5]:
!ls


drive  sample_data  spark-3.5.0-bin-hadoop3.tgz


In [6]:
!tar xf spark-3.5.0-bin-hadoop3.tgz

Además de la instalación habitual, hay un paso más que debemos dar, y es la instalación de la librería *findspark*. Esta librería nos permitirá encontrar la instalación de Apache Spark en nuestro sistema.

Establecemos también un par de variables de entorno.

In [7]:
!pip install -q findspark

In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [9]:
import findspark
findspark.init()

In [10]:
findspark.find()

'/content/spark-3.5.0-bin-hadoop3'

Una vez tenemos instalado nuestro framework, es momento de crear nuestra variable SparkSession, de la cual parten la mayoría de funcionalidades de SparkSQL

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Pyspark_SQL")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [12]:
spark

### Lectura de datos

Ahora sí, podemos comenzar a trabajar con SparkSQL. Comenzaremos leyendo nuestro conjunto de datos, el cual contiene información de valoraciones de restaurantes.

In [13]:
!pwd

/content


In [14]:
import pandas as pd

data_path = '/content/drive/MyDrive/TokioSchool/'

data = spark.read.options(inferSchema='True',delimiter=';', header=True).csv (data_path + 'StudentPerformance.csv')


In [15]:
data

DataFrame[index: int, race_ethnicity: string, parental_education: string, lunch: string, test_preparation_course: string, math_percentage: double, reading_percentage: double, writing_percentage: double, sex: string]

### Análisis estadísico

Para usar solo algunas columnas podemos usar el método *select*

También podemos sacar algunas estadísticas concretas como:

*   a) Puntuación media por etnia y asignatura
*   b) Puntuación media según el nivel educativo de los padres por asignatura
*   c) Puntuación media por sexo y asignatura
*   d) Puntuación media por tipo de almuerzo y asignatura



In [25]:
# a1) Número de etnias distintas
data.select('race_ethnicity').distinct().count()

5

In [26]:
# a2) Puntuación media por étnia en lectura
data.groupby('race_ethnicity').agg({'reading_percentage': 'mean'}).collect()

[Row(race_ethnicity='group B', avg(reading_percentage)=0.6735263157894739),
 Row(race_ethnicity='group C', avg(reading_percentage)=0.6910344827586206),
 Row(race_ethnicity='group D', avg(reading_percentage)=0.7003053435114501),
 Row(race_ethnicity='group A', avg(reading_percentage)=0.6467415730337079),
 Row(race_ethnicity='group E', avg(reading_percentage)=0.7302857142857146)]

Podemos ver cómo en el grupo A, la media es de 0.647, la media del grupo B es de 0.674, la media del grupo C es de 0.691, la media del grupo D es de 0.7, y la media del grupo E es de 0.73. Se puede deducir que la media del grupo étnico E es superior en lectura que el resto de etnias.

In [27]:
# a3) Puntuación media por étnia en matemáticas
data.groupby('race_ethnicity').agg({'math_percentage': 'mean'}).collect()

[Row(race_ethnicity='group B', avg(math_percentage)=0.6345263157894734),
 Row(race_ethnicity='group C', avg(math_percentage)=0.644639498432602),
 Row(race_ethnicity='group D', avg(math_percentage)=0.6736259541984732),
 Row(race_ethnicity='group A', avg(math_percentage)=0.6162921348314606),
 Row(race_ethnicity='group E', avg(math_percentage)=0.7382142857142858)]

Podemos ver cómo en el grupo A, la media es de 0.617, la media del grupo B es de 0.635, la media del grupo C es de 0.645, la media del grupo D es de 0.674, y la media del grupo E es de 0.738. Se puede deducir que la media del grupo étnico E es superior en matemáticas que el resto de etnias.

In [28]:
# a4) Puntuación media por étnia en escritura
data.groupby('race_ethnicity').agg({'writing_percentage': 'mean'}).collect()

[Row(race_ethnicity='group B', avg(writing_percentage)=0.6560000000000001),
 Row(race_ethnicity='group C', avg(writing_percentage)=0.6782758620689658),
 Row(race_ethnicity='group D', avg(writing_percentage)=0.7014503816793896),
 Row(race_ethnicity='group A', avg(writing_percentage)=0.6267415730337079),
 Row(race_ethnicity='group E', avg(writing_percentage)=0.7140714285714282)]

Podemos ver cómo en el grupo A, la media es de 0.627, la media del grupo B es de 0.656, la media del grupo C es de 0.678, la media del grupo D es de 0.701, y la media del grupo E es de 0.714. Se puede deducir que la media del grupo étnico E es superior en escritura que el resto de etnias.

Se puede deducir que la media del grupo E es superior en todas las materias.

In [55]:
# b1) niveles educativos
data.select('parental_education').distinct().show()


+------------------+
|parental_education|
+------------------+
|  some high school|
|associate's degree|
|       high school|
| bachelor's degree|
|   master's degree|
|      some college|
+------------------+



In [31]:
# b2) Puntuación media de los hijos en escritura por nivel educativo de los padres
data.groupby('parental_education').agg({'writing_percentage': 'mean'}).collect()

[Row(parental_education='some high school', avg(writing_percentage)=0.6488826815642459),
 Row(parental_education="associate's degree", avg(writing_percentage)=0.698963963963964),
 Row(parental_education='high school', avg(writing_percentage)=0.6244897959183671),
 Row(parental_education="bachelor's degree", avg(writing_percentage)=0.733813559322034),
 Row(parental_education="master's degree", avg(writing_percentage)=0.7567796610169494),
 Row(parental_education='some college', avg(writing_percentage)=0.6884070796460179)]

La media más alta en escritura la obtienen los estudiantes cuyos padres tienen los niveles de estudios más altos, los que tienen master 0.756 y los que tienen carrera con una media de 0.734

In [32]:
# b3) Puntuación media de los hijos en matemáticas por nivel educativo de los padres
data.groupby('parental_education').agg({'math_percentage': 'mean'}).collect()

[Row(parental_education='some high school', avg(math_percentage)=0.6349720670391062),
 Row(parental_education="associate's degree", avg(math_percentage)=0.6788288288288293),
 Row(parental_education='high school', avg(math_percentage)=0.6213775510204079),
 Row(parental_education="bachelor's degree", avg(math_percentage)=0.6938983050847456),
 Row(parental_education="master's degree", avg(math_percentage)=0.6974576271186439),
 Row(parental_education='some college', avg(math_percentage)=0.6712831858407079)]

La media más alta en matemáticas la obtienen los estudiantes cuyos padres tienen los niveles de estudios más altos, los que tienen master 0.697 y los que tienen carrera con una media de 0.694

In [37]:
# b4) Puntuación media de los hijos en lectura por nivel educativo de los padres
data.groupby('parental_education').agg({'reading_percentage': 'mean'}).collect()

[Row(parental_education='some high school', avg(reading_percentage)=0.6693854748603356),
 Row(parental_education="associate's degree", avg(reading_percentage)=0.7092792792792795),
 Row(parental_education='high school', avg(reading_percentage)=0.6470408163265308),
 Row(parental_education="bachelor's degree", avg(reading_percentage)=0.7300000000000003),
 Row(parental_education="master's degree", avg(reading_percentage)=0.753728813559322),
 Row(parental_education='some college', avg(reading_percentage)=0.6946017699115047)]

La media más alta en escritura la obtienen los estudiantes cuyos padres tienen los niveles de estudios más altos, los que tienen master 0.734 y los que tienen carrera con una media de 0.73

Se puede deducir que el nivel educativo de los padres afecta directamente a las notas de sus hijos, ya que a más nivel educativo, mejores serán las notas de media en los tres aspectos, lectura, escritura y matemáticas.

In [49]:
# c1) Puntuación media de los hijos por sexo en matemáticas
data.groupby('sex').agg({'math_percentage': 'mean'}).collect()

[Row(sex='F', avg(math_percentage)=0.636332046332046),
 Row(sex='M', avg(math_percentage)=0.6872821576763488)]

La puntuación media en matemáticas de media será mayor en hombres

In [51]:
# c2) Puntuación media de los hijos por sexo en lectura
data.groupby('sex').agg({'reading_percentage': 'mean'}).collect()

[Row(sex='F', avg(reading_percentage)=0.7260810810810804),
 Row(sex='M', avg(reading_percentage)=0.6547302904564317)]

La puntuación media en lectura de media será mayor en mujeres

In [52]:
# c3) Puntuación media de los hijos por sexo en escritura
data.groupby('sex').agg({'writing_percentage': 'mean'}).collect()

[Row(sex='F', avg(writing_percentage)=0.7246718146718146),
 Row(sex='M', avg(writing_percentage)=0.6331120331950212)]

La puntuación media en escritura de media será mayor en mujeres

No se puede concluir que el sexo sea una variable determinante en la media de los alumnos de forma global ya que depende de las asignaturas a las que hagamos referencia

In [56]:
# d1.1) ¿Cuál es la puntuación media en matemáticas con lo almuerzo estandar?
data.filter(data.lunch == "standard").agg({'math_percentage' : 'mean'}).show()

+--------------------+
|avg(math_percentage)|
+--------------------+
|   0.700341085271318|
+--------------------+



In [57]:
# d1.2) ¿Cuál es la puntuación media en matemáticas con lo almuerzo free o reduced?
data.filter(data.lunch == "free/reduced").agg({'math_percentage' : 'mean'}).show()

+--------------------+
|avg(math_percentage)|
+--------------------+
|  0.5892112676056344|
+--------------------+



In [61]:
# d2) Puntuación media de los hijos por tipo de almuerzo en escritura
data.groupby('lunch').agg({'writing_percentage': 'mean'}).collect()

[Row(lunch='free/reduced', avg(writing_percentage)=0.6302253521126763),
 Row(lunch='standard', avg(writing_percentage)=0.7082325581395355)]

In [62]:
# d2) Puntuación media de los hijos por tipo de almuerzo en lectura
data.groupby('lunch').agg({'reading_percentage': 'mean'}).collect()

[Row(lunch='free/reduced', avg(reading_percentage)=0.6465352112676062),
 Row(lunch='standard', avg(reading_percentage)=0.7165426356589147)]

Los alumnos con almuerzo free o reduced obtienen de media peores notas en todas las asignaturas que los alumnos que obtienen un almuerzo estandar.